In [1]:
import numpy as np
import pandas as pd
import itertools as it
import heapq as hq
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("C:/Users/Tobias Giesgen/PycharmProjects/gempy")
sys.path.append("../")
import bayseg
import gempy as gp

In [2]:
data_s3 = pd.read_csv("../data/GemPy_models/Synthetic_well_data/gempy_model_4layers_horizontal_s4_75.csv")

In [22]:
a = data_s3.drop_duplicates(subset='Well Name').reset_index()

In [23]:
points = []
for k in range(len(a)):
    points.append((a.loc[k, 'X'], a.loc[k, 'Y']))

In [123]:
distances = []
nbh_dis = []
nbh = []
midpoints = []

for j in range(len(points)):
    distances += [[euclideanDistance(points[j], points[i]) for i in range(len(points))]] # stores distances between two points
    nbh_dis.append(hq.nsmallest(5, distances[j])[1:])
    nbh.append([np.where(distances[j] == nbh_dis[j][k])[0][0] for k in range(len(nbh_dis[j]))])
    midpoints.append([find_midpoint(a.X.loc[j], a.Y.loc[j], 
                                   a.X.loc[nbh[j][i]], a.Y.loc[nbh[j][i]]) for i in range(4)])
    

6.82 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
coords = data_s3.loc[:,['X','Y','Z']].values

In [125]:
coords

array([[ 145.3 ,  115.3 ,   -0.52],
       [ 145.3 ,  115.3 ,   -1.58],
       [ 145.3 ,  115.3 ,   -2.62],
       ...,
       [ 151.3 ,  144.7 , -196.88],
       [ 151.3 ,  144.7 , -197.92],
       [ 151.3 ,  144.7 , -198.98]])

In [120]:
midpoints[4]

[[138.7, 61.900000000000006],
 [117.39999999999999, 65.8],
 [122.8, 72.4],
 [137.5, 73.9]]

In [121]:
# nearest boreholes to borehole 0
a.loc[nbh[4]]

,index,X,Y,Z,layer,Well Name,feature1,feature2,feature3,feature4
3,570,147.7,75.7,-0.52,3,BH4,-8.01,10.63,-2.79,4.74
7,1330,105.1,83.5,-0.52,3,BH8,-12.37,3.35,-13.69,4.35
6,1140,115.9,96.7,-0.52,3,BH7,-2.89,1.86,-6.03,10.37
5,950,145.3,99.7,-0.52,3,BH6,-3.07,5.46,-2.26,-1.02


In [122]:
a.loc[4].values

array([760, 129.7, 48.1, -0.52, 3, 'BH5', -8.18, 12.1, -3.75, 14.01],
      dtype=object)

In [92]:
midpoints = []
for i in range(4):
    midpoints.append(find_midpoint(a.X.loc[0], a.Y.loc[0], 
                                   a.X.loc[nbh[0][i]], a.Y.loc[nbh[0][i]]))

In [93]:
midpoints

[<generator object <genexpr> at 0x000002B16A65F2B0>]

In [82]:
def find_midpoint(point1_x, point1_y, point2_x, point2_y):
    X = (point1_x + point2_x)/2
    Y = (point1_y + point2_y)/2
    return [X,Y]

In [45]:
distances[1]

[75.17978451685,
 0.0,
 61.38794669965758,
 82.02243595504828,
 81.47637694448618,
 74.55467792164353,
 45.670997361564154,
 40.13277961965753,
 11.462983904725684,
 89.88993269549154]

In [26]:
def euclideanDistance(coordinate1, coordinate2):
    return pow(pow(coordinate1[0] - coordinate2[0], 2) + pow(coordinate1[1] - coordinate2[1], 2), .5)

In [5]:
def nearest_points(self, layer):
    lst = self.geo_data.interfaces.loc[np.where(self.geo_data.interfaces.formation_number == layer)[0]]
    lst = lst.reset_index(drop=True)

    # store points pairwise with their x and y coordinate
    points = []
    for k in range(len(lst)):
        points.append((lst.loc[k, 'X'], lst.loc[k, 'Y']))

    # calculate distances and storage with correspondig position
    distances = []
    pos = []
    for i in range(len(points) - 1):
        for j in range(i + 1, len(points)):
            distances += [euclideanDistance(points[i], points[j])]  # stores distances between two points
            pos += [[i, j]]  # stores corresponding points

    # gives points with smallest distance
    final_points = pos[np.where(distances == min(distances))[0][0]]

    X = (lst.X.loc[final_points[0]] + lst.X.loc[final_points[1]]) / 2
    Y = (lst.Y.loc[final_points[0]] + lst.Y.loc[final_points[1]]) / 2

    # append mean point between the two nearest points
    points.append((X, Y))

    # calculate distance between mean point and all other values
    distances = []
    pos = []
    for i in range(len(points) - 1):
        distances += [euclideanDistance(points[i], points[-1])]  # stores distances between two points
        pos += [i]  # stores correspondig points

    # points which is nearest to mean point
    k = hq.nsmallest(3, distances)[2]
    #m = hq.nsmallest(4, distances)[3]

    # 3 nearest points in lst
    final_points.append(np.where(distances == k)[0][0])
    #final_points.append(np.where(distances == m)[0][0])

    u = np.sum(lst.loc[final_points, ['X', 'Y', 'Z','formation_number']], axis=1).values
    p = np.sum(self.geo_data.interfaces.loc[:, ['X', 'Y', 'Z','formation_number']], axis=1)
    p = p.sort_index()

    final_points2 = []
    for i in range(len(u)):
        final_points2.append(np.where(p == u[i])[0][0])



    return final_points2